# 7. Translation

**Estimated Time**: ~2 hours

**Prerequisites**: Notebooks 1-6 (especially Notebook 4: Text Summarization for encoder-decoder concepts)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** how sequence-to-sequence models convert text between languages
2. **Explain** encoder-decoder architecture for translation tasks
3. **Work with** different language pairs and multilingual models
4. **Evaluate** translation quality conceptually (BLEU scores)
5. **Build** a multilingual content adapter for real-world translation tasks

## Setup

Run this cell first. If you completed previous notebooks, you already have the core packages ready.

In [ ]:
# Core imports
from transformers import pipeline

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("Setup complete!")

---

# Part 1: Conceptual Foundation

## What is Machine Translation?

**In plain English**: Machine translation automatically converts text from one language to another, preserving meaning while adapting to the target language's grammar and style.

**Technical definition**: Neural Machine Translation (NMT) uses encoder-decoder transformer architectures to map input sequences in a source language to output sequences in a target language, learning complex linguistic patterns from parallel text corpora.

### The Evolution of Machine Translation

```
TRANSLATION HISTORY:
┌────────────────────────────────────────────────────────────────┐
│  1950s-1980s: RULE-BASED                                      │
│  ├── Hand-crafted grammar rules                               │
│  ├── Dictionaries for word-by-word translation                │
│  └── Brittle, couldn't handle nuance                          │
│                                                                │
│  1990s-2010s: STATISTICAL                                     │
│  ├── Learned patterns from parallel texts                     │
│  ├── Phrase-based translation                                 │
│  └── Better but still struggled with long sentences           │
│                                                                │
│  2014-Present: NEURAL (What we use today!)                    │
│  ├── End-to-end learning                                      │
│  ├── Attention mechanisms capture context                     │
│  └── Near-human quality for many language pairs               │
└────────────────────────────────────────────────────────────────┘
```

### How It Works: Encoder-Decoder for Translation

Translation uses the same encoder-decoder architecture you learned in Notebook 4 (Summarization):

```
ENCODER-DECODER TRANSLATION:

Source (English):  "The cat sits on the mat."
                          │
                          ▼
┌─────────────────────────────────────────────────────────────┐
│                       ENCODER                               │
│  ┌─────────────────────────────────────────────────────┐   │
│  │ Tokenize: ["The", "cat", "sits", "on", "the", "mat"] │   │
│  │           ↓    ↓     ↓     ↓    ↓     ↓              │   │
│  │ Embed:   [v1,  v2,   v3,   v4,  v5,   v6]            │   │
│  │           ↓    ↓     ↓     ↓    ↓     ↓              │   │
│  │ Self-Attention layers (understand relationships)     │   │
│  │           ↓                                          │   │
│  │ Context Vectors (meaning representation)             │   │
│  └─────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────┘
                          │
                          ▼ (meaning passes to decoder)
┌─────────────────────────────────────────────────────────────┐
│                       DECODER                               │
│  ┌─────────────────────────────────────────────────────┐   │
│  │ Cross-Attention to encoder outputs                   │   │
│  │           ↓                                          │   │
│  │ Generate one token at a time in target language      │   │
│  │ "Le" → "chat" → "est" → "assis" → "sur" → ...       │   │
│  │           ↓                                          │   │
│  │ Until <end> token                                    │   │
│  └─────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────┘
                          │
                          ▼
Target (French):  "Le chat est assis sur le tapis."
```

**Key insight**: The encoder captures the *meaning* of the source text, and the decoder expresses that meaning in the target language.

### Connection to Notebook 4 (Summarization)

| Aspect | Summarization | Translation |
|--------|---------------|-------------|
| Architecture | Encoder-Decoder | Encoder-Decoder |
| Input | Long text | Source language text |
| Output | Shorter text (same language) | Target language text |
| Transformation | Compression | Language conversion |
| Key challenge | Preserving meaning while shortening | Preserving meaning across languages |

Both tasks use the same fundamental architecture but solve different problems!

### Language Pairs and Directionality

```
LANGUAGE PAIRS:

BILINGUAL MODELS (one direction):
┌─────────────────────────────────────────────────────────────┐
│  English → French  (en-fr)                                 │
│  French → English  (fr-en)  ← Separate model needed!       │
│  English → German  (en-de)                                 │
│  German → English  (de-en)  ← Separate model needed!       │
└─────────────────────────────────────────────────────────────┘

MULTILINGUAL MODELS (many directions):
┌─────────────────────────────────────────────────────────────┐
│              ┌─────────────┐                               │
│  English ────┤             ├──── French                    │
│  German  ────┤  ONE MODEL  ├──── Spanish                   │
│  Chinese ────┤             ├──── Japanese                  │
│  ...     ────┤  (100+ languages) ├──── ...                 │
│              └─────────────┘                               │
│                                                            │
│  Example: Helsinki-NLP/opus-mt-mul, mBART, M2M-100        │
└─────────────────────────────────────────────────────────────┘
```

### Real-World Applications

Machine translation powers:

- **Communication**: Real-time conversation translation (Google Translate, DeepL)
- **Business**: Localizing websites, apps, and marketing materials
- **Research**: Accessing scientific papers in other languages
- **Media**: Subtitles, dubbing, and content localization
- **E-commerce**: Translating product listings for global markets
- **Customer Support**: Multilingual chatbots and ticket handling

### Key Terminology

| Term | Definition |
|------|------------|
| **Source language** | The original language of the input text |
| **Target language** | The language you're translating into |
| **Language pair** | Source-target combination (e.g., en-fr) |
| **Parallel corpus** | Training data with aligned translations |
| **BLEU score** | Metric comparing translation to human references |
| **Back-translation** | Translating output back to source for quality check |
| **Multilingual model** | Single model handling many language pairs |

### Check Your Understanding

Before moving on, try to answer these questions (answers at the end):

1. What architecture do modern translation models use?
   - A) Encoder-only (like BERT)
   - B) Decoder-only (like GPT)
   - C) Encoder-decoder (like T5, BART)

2. What is the role of the encoder in translation?
   - A) Generate target language tokens
   - B) Capture the meaning of source text
   - C) Calculate the BLEU score

3. What is a "language pair"?
   - A) Two words that mean the same thing
   - B) A source and target language combination
   - C) Two different translation models

4. What is back-translation used for?
   - A) Training new models
   - B) Quality checking translations
   - C) Both A and B

---

# Part 2: Basic Implementation

## Your First Translation

Let's translate text using the Hugging Face pipeline:

In [ ]:
# Create a translation pipeline (English to French)
# Using Helsinki-NLP models - lightweight and effective
translator_en_fr = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

# Translate a simple sentence
text = "Hello, how are you today?"
result = translator_en_fr(text)

print("English to French Translation:")
print("="*50)
print(f"English: {text}")
print(f"French:  {result[0]['translation_text']}")

### Understanding the Output

The pipeline returns a list of dictionaries with:
- `translation_text`: The translated text

In [ ]:
# Examine the full output structure
print("Full Output Structure:")
print("="*50)
print(result)
print(f"\nType: {type(result)}")
print(f"Length: {len(result)}")
print(f"First element keys: {result[0].keys()}")

### Translating Multiple Sentences

In [ ]:
# Translate multiple sentences at once (batching)
sentences = [
    "The weather is beautiful today.",
    "I love learning new languages.",
    "Machine translation has improved dramatically.",
    "Can you help me find the train station?",
]

results = translator_en_fr(sentences)

print("Batch Translation (English → French):")
print("="*60)

for eng, result in zip(sentences, results):
    print(f"\nEN: {eng}")
    print(f"FR: {result['translation_text']}")

### Different Language Pairs

Let's try translating to different languages:

In [ ]:
# Load translators for different language pairs
print("Loading translation models...")
translator_en_de = pipeline("translation", model="Helsinki-NLP/opus-mt-en-de")  # English → German
translator_en_es = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")  # English → Spanish
print("Models loaded!")

In [ ]:
# Translate the same text to multiple languages
text = "Artificial intelligence is transforming how we live and work."

print(f"Original (English):\n  {text}")
print("\n" + "="*60)

translations = {
    "French": translator_en_fr(text)[0]['translation_text'],
    "German": translator_en_de(text)[0]['translation_text'],
    "Spanish": translator_en_es(text)[0]['translation_text'],
}

for lang, trans in translations.items():
    print(f"\n{lang}:\n  {trans}")

### Reverse Translation (Target → Source)

In [ ]:
# Load reverse direction translator
translator_fr_en = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")

# Translate French to English
french_text = "Bonjour, je suis très content de vous rencontrer."
result = translator_fr_en(french_text)

print("French to English Translation:")
print("="*50)
print(f"French:  {french_text}")
print(f"English: {result[0]['translation_text']}")

---

## Exercise 1: Multi-Language Translator (Guided)

**Difficulty**: Basic | **Time**: 10-15 minutes

**Your task**: Build a function that translates text to multiple languages at once.

### Step 1: Create a multi-language translator function

In [ ]:
def translate_to_multiple(text, translators_dict):
    """
    Translate text to multiple languages.
    
    Args:
        text: The text to translate (in English)
        translators_dict: Dict of {language_name: translator_pipeline}
        
    Returns:
        dict with translations for each language
    """
    results = {
        'original': text,
        'translations': {}
    }
    
    for lang_name, translator in translators_dict.items():
        translation = translator(text)[0]['translation_text']
        results['translations'][lang_name] = translation
    
    return results


# Set up translators
translators = {
    'French': translator_en_fr,
    'German': translator_en_de,
    'Spanish': translator_en_es,
}

# Test with sample texts
test_texts = [
    "Welcome to our online store!",
    "Your order has been shipped.",
    "Thank you for your purchase.",
]

print("Multi-Language Translation:")
print("="*60)

for text in test_texts:
    result = translate_to_multiple(text, translators)
    
    print(f"\nOriginal: {result['original']}")
    for lang, trans in result['translations'].items():
        print(f"  {lang:10s}: {trans}")

### Step 2: Add a display formatter

In [ ]:
def format_translation_result(result):
    """Format translation results for nice display."""
    lines = []
    lines.append("┌" + "─"*58 + "┐")
    lines.append(f"│ Original: {result['original'][:45]:45s} │")
    lines.append("├" + "─"*58 + "┤")
    
    for lang, trans in result['translations'].items():
        # Truncate if too long
        trans_display = trans[:43] + '...' if len(trans) > 45 else trans
        lines.append(f"│ {lang:8s}: {trans_display:46s} │")
    
    lines.append("└" + "─"*58 + "┘")
    return '\n'.join(lines)


# Test the formatter
text = "Machine learning is revolutionizing many industries."
result = translate_to_multiple(text, translators)

print(format_translation_result(result))

### Step 3: Try your own sentences

In [ ]:
# YOUR CODE HERE
# Write your own sentences and translate them

my_text = "Your sentence here"

# Uncomment to run:
# result = translate_to_multiple(my_text, translators)
# print(format_translation_result(result))

---

# Part 3: Intermediate Exploration

## Translation Parameters

Like text generation, translation models support parameters to control output:

In [ ]:
# Explore generation parameters
text = "The new smartphone features an innovative camera system with artificial intelligence."

# Default translation
default_result = translator_en_fr(text)

# With beam search (explores multiple paths)
beam_result = translator_en_fr(
    text,
    num_beams=5,  # Consider 5 candidate translations
    early_stopping=True
)

# With sampling (more variety)
sample_result = translator_en_fr(
    text,
    do_sample=True,
    temperature=0.7,
    top_k=50,
)

print(f"Original: {text}")
print("="*70)
print(f"\nDefault:  {default_result[0]['translation_text']}")
print(f"\nBeam(5):  {beam_result[0]['translation_text']}")
print(f"\nSampled:  {sample_result[0]['translation_text']}")

### Parameter Guide for Translation

| Parameter | Effect | Typical Value |
|-----------|--------|---------------|
| `num_beams` | More beams = better quality, slower | 4-5 for quality |
| `do_sample` | Enable random sampling | False for consistency |
| `temperature` | Randomness (if sampling) | 0.7-1.0 |
| `max_length` | Maximum output tokens | Depends on input |
| `length_penalty` | Encourage shorter/longer output | 1.0 (neutral) |

## Handling Idioms and Cultural Expressions

Idioms are challenging because literal translation loses meaning:

In [ ]:
# Test with idioms
idioms = [
    "It's raining cats and dogs.",
    "Break a leg!",
    "That costs an arm and a leg.",
    "Let's hit the road.",
    "She's feeling under the weather.",
]

print("Idiom Translation Test (English → French):")
print("="*60)
print("Note: Idioms often lose meaning in literal translation.\n")

for idiom in idioms:
    result = translator_en_fr(idiom)[0]['translation_text']
    print(f"EN: {idiom}")
    print(f"FR: {result}")
    print()

### Back-Translation for Quality Checking

Back-translation translates output back to the source language to verify meaning preservation:

In [ ]:
def back_translate(text, forward_translator, backward_translator):
    """
    Translate text and then translate it back to check quality.
    
    Args:
        text: Original text
        forward_translator: Source → Target translator
        backward_translator: Target → Source translator
        
    Returns:
        dict with original, translation, and back-translation
    """
    # Forward translation
    translation = forward_translator(text)[0]['translation_text']
    
    # Back translation
    back_translation = backward_translator(translation)[0]['translation_text']
    
    return {
        'original': text,
        'translation': translation,
        'back_translation': back_translation,
    }


# Test back-translation
test_sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "I would like to book a table for two at 7 PM.",
    "This product comes with a two-year warranty.",
]

print("Back-Translation Quality Check (EN → FR → EN):")
print("="*60)

for sentence in test_sentences:
    result = back_translate(sentence, translator_en_fr, translator_fr_en)
    
    # Check similarity
    words_original = set(result['original'].lower().split())
    words_back = set(result['back_translation'].lower().split())
    overlap = len(words_original & words_back) / len(words_original | words_back)
    
    print(f"\nOriginal:    {result['original']}")
    print(f"French:      {result['translation']}")
    print(f"Back to EN:  {result['back_translation']}")
    print(f"Word overlap: {overlap:.1%}")

### Understanding BLEU Scores (Conceptual)

BLEU (Bilingual Evaluation Understudy) is the standard metric for translation quality:

```
BLEU SCORE EXPLANATION:

Measures n-gram overlap between machine translation and human reference(s)

Example:
Reference:    "The cat sits on the mat."
Machine:      "The cat is on the mat."

1-gram matches: "The", "cat", "on", "the", "mat" (5/6 words)
2-gram matches: "The cat", "on the", "the mat" (3/5 bigrams)
... and so on for 3-grams, 4-grams

BLEU = geometric mean of n-gram precisions × brevity penalty

Score interpretation:
├── <10:  Almost useless
├── 10-20: Some meaning preserved  
├── 20-30: Understandable but rough
├── 30-40: Good quality
├── 40-50: High quality
└── 50+:   Very high quality (human-like)
```

In [ ]:
# Simple BLEU-like demonstration (not the real algorithm, but shows the concept)
def simple_overlap_score(reference, candidate):
    """
    Calculate a simple word overlap score (not real BLEU).
    This demonstrates the concept of comparing translations.
    """
    ref_words = reference.lower().replace('.', '').replace(',', '').split()
    cand_words = candidate.lower().replace('.', '').replace(',', '').split()
    
    # 1-gram overlap
    ref_set = set(ref_words)
    cand_set = set(cand_words)
    
    matches = len(ref_set & cand_set)
    total = len(ref_set | cand_set)
    
    return matches / total if total > 0 else 0


# Example: Compare back-translation to original
original = "The new product launch exceeded our expectations."
result = back_translate(original, translator_en_fr, translator_fr_en)

score = simple_overlap_score(original, result['back_translation'])

print("Translation Quality Estimation:")
print("="*50)
print(f"Original:        {original}")
print(f"Back-translated: {result['back_translation']}")
print(f"\nOverlap score: {score:.1%}")
print("(Higher = better meaning preservation)")

---

## Exercise 2: Idiom and Back-Translation Analyzer (Semi-guided)

**Difficulty**: Intermediate | **Time**: 15-20 minutes

**Your task**: Build a function that:
1. Translates text to a target language
2. Back-translates to check quality
3. Flags potential issues with idioms or complex phrases

**Hints**:
- Use back-translation to detect meaning loss
- Compare word overlap before and after
- Flag translations where back-translation differs significantly

In [ ]:
# YOUR CODE HERE

def analyze_translation_quality(text, forward_translator, backward_translator, 
                                similarity_threshold=0.6):
    """
    Analyze translation quality using back-translation.
    
    Args:
        text: Original text to translate
        forward_translator: Source → Target pipeline
        backward_translator: Target → Source pipeline
        similarity_threshold: Minimum acceptable similarity
        
    Returns:
        dict with analysis results and warnings
    """
    # Forward translation
    translation = forward_translator(text)[0]['translation_text']
    
    # Back translation
    back_translation = backward_translator(translation)[0]['translation_text']
    
    # Calculate similarity
    original_words = set(text.lower().replace('.', '').replace(',', '').replace('!', '').split())
    back_words = set(back_translation.lower().replace('.', '').replace(',', '').replace('!', '').split())
    
    intersection = len(original_words & back_words)
    union = len(original_words | back_words)
    similarity = intersection / union if union > 0 else 0
    
    # Detect potential issues
    warnings = []
    
    if similarity < similarity_threshold:
        warnings.append(f"Low back-translation similarity ({similarity:.1%})")
    
    # Check for words that might be idioms (multi-word expressions that changed a lot)
    lost_words = original_words - back_words
    if len(lost_words) > len(original_words) * 0.3:
        warnings.append(f"Many words not preserved: {lost_words}")
    
    return {
        'original': text,
        'translation': translation,
        'back_translation': back_translation,
        'similarity': similarity,
        'quality': 'Good' if similarity >= similarity_threshold else 'Review Needed',
        'warnings': warnings,
    }


# Test with various sentences
test_cases = [
    "Our company offers excellent customer service.",  # Straightforward
    "It's raining cats and dogs outside.",  # Idiom
    "The meeting is scheduled for 3 PM tomorrow.",  # Business language
    "You really knocked it out of the park!",  # Idiom
    "Please review the attached document.",  # Formal
]

print("Translation Quality Analysis:")
print("="*70)

for text in test_cases:
    result = analyze_translation_quality(text, translator_en_fr, translator_fr_en)
    
    quality_icon = "✓" if result['quality'] == 'Good' else "⚠"
    
    print(f"\n{quality_icon} Original: {result['original']}")
    print(f"  French:  {result['translation']}")
    print(f"  Back:    {result['back_translation']}")
    print(f"  Quality: {result['quality']} (similarity: {result['similarity']:.1%})")
    
    if result['warnings']:
        for warning in result['warnings']:
            print(f"  Warning: {warning}")

---

# Part 4: Advanced Topics

## Multilingual Models

Some models can translate between many language pairs:

In [ ]:
# Many-to-many translation with NLLB (No Language Left Behind)
# Note: This is a larger model and may take longer to load
print("Available Helsinki-NLP models for various language pairs:")
print("="*60)

helsinki_models = [
    ("en-fr", "English → French"),
    ("en-de", "English → German"),
    ("en-es", "English → Spanish"),
    ("en-it", "English → Italian"),
    ("en-pt", "English → Portuguese"),
    ("en-nl", "English → Dutch"),
    ("en-ru", "English → Russian"),
    ("en-zh", "English → Chinese"),
    ("en-ja", "English → Japanese"),
    ("en-ar", "English → Arabic"),
]

for code, description in helsinki_models:
    model_name = f"Helsinki-NLP/opus-mt-{code}"
    print(f"  {code:8s} {description:25s} → {model_name}")

In [ ]:
# Load a few more translators to demonstrate variety
print("Loading Italian translator...")
translator_en_it = pipeline("translation", model="Helsinki-NLP/opus-mt-en-it")

# Test with a product description
product_text = "This wireless earbuds feature noise cancellation and 24-hour battery life."

print(f"\nProduct Description Translation:")
print("="*60)
print(f"English: {product_text}")
print(f"\nFrench:  {translator_en_fr(product_text)[0]['translation_text']}")
print(f"German:  {translator_en_de(product_text)[0]['translation_text']}")
print(f"Spanish: {translator_en_es(product_text)[0]['translation_text']}")
print(f"Italian: {translator_en_it(product_text)[0]['translation_text']}")

### Domain-Specific Considerations

Translation quality varies by domain:

In [ ]:
# Test different domains
domain_texts = {
    "General": "The weather is nice today.",
    "Technical": "The API endpoint returns a JSON response with status codes.",
    "Legal": "The party of the first part hereby agrees to the terms and conditions.",
    "Medical": "The patient presents with acute respiratory distress syndrome.",
    "Marketing": "Our revolutionary product will transform your daily routine!",
}

print("Domain-Specific Translation (English → French):")
print("="*60)
print("Note: General models may struggle with specialized vocabulary.\n")

for domain, text in domain_texts.items():
    result = translator_en_fr(text)[0]['translation_text']
    print(f"[{domain}]")
    print(f"  EN: {text}")
    print(f"  FR: {result}")
    print()

### Limitations of Machine Translation

| Limitation | Description | Example |
|------------|-------------|--------|
| **Idioms** | Literal translation loses meaning | "It's raining cats and dogs" |
| **Context** | Missing broader context affects quality | Pronouns without antecedents |
| **Cultural references** | Culture-specific content may not translate | Holidays, customs |
| **Ambiguity** | Words with multiple meanings | "Bank" (financial vs. river) |
| **Rare languages** | Less training data = lower quality | Indigenous languages |
| **Specialized domains** | Technical vocabulary may be unknown | Medical, legal jargon |

---

## Exercise 3: Translation Quality Comparison (Independent)

**Difficulty**: Advanced | **Time**: 15-20 minutes

**Your task**: Build a class that compares translation quality across different language pairs using back-translation analysis.

**Requirements**:
1. Support multiple target languages
2. Use back-translation to estimate quality
3. Rank languages by translation fidelity for given input

In [ ]:
# YOUR CODE HERE

class TranslationQualityComparer:
    """
    Compares translation quality across multiple language pairs.
    """
    
    def __init__(self):
        # Store translator pairs
        self.language_pairs = {}
    
    def add_language_pair(self, language_name, forward_translator, backward_translator):
        """
        Add a language pair for comparison.
        """
        self.language_pairs[language_name] = {
            'forward': forward_translator,
            'backward': backward_translator,
        }
    
    def calculate_similarity(self, text1, text2):
        """
        Calculate word-level similarity between two texts.
        """
        words1 = set(text1.lower().replace('.', '').replace(',', '').replace('!', '').replace('?', '').split())
        words2 = set(text2.lower().replace('.', '').replace(',', '').replace('!', '').replace('?', '').split())
        
        intersection = len(words1 & words2)
        union = len(words1 | words2)
        
        return intersection / union if union > 0 else 0
    
    def compare_translations(self, text):
        """
        Translate text to all languages and compare quality.
        
        Returns:
            dict with results for each language, ranked by quality
        """
        results = []
        
        for lang_name, translators in self.language_pairs.items():
            # Translate forward
            translation = translators['forward'](text)[0]['translation_text']
            
            # Translate back
            back_translation = translators['backward'](translation)[0]['translation_text']
            
            # Calculate similarity
            similarity = self.calculate_similarity(text, back_translation)
            
            results.append({
                'language': lang_name,
                'translation': translation,
                'back_translation': back_translation,
                'similarity': similarity,
            })
        
        # Sort by similarity (highest first)
        results.sort(key=lambda x: x['similarity'], reverse=True)
        
        return {
            'original': text,
            'results': results,
            'best_language': results[0]['language'] if results else None,
        }
    
    def format_comparison(self, comparison):
        """
        Format comparison results for display.
        """
        lines = []
        lines.append(f"Original: {comparison['original']}")
        lines.append("="*70)
        lines.append(f"Best translation quality: {comparison['best_language']}")
        lines.append("")
        
        for i, result in enumerate(comparison['results'], 1):
            medal = ["🥇", "🥈", "🥉"][i-1] if i <= 3 else f"{i}."
            lines.append(f"{medal} {result['language']} (similarity: {result['similarity']:.1%})")
            lines.append(f"   Translation: {result['translation'][:50]}..." if len(result['translation']) > 50 else f"   Translation: {result['translation']}")
            lines.append(f"   Back:        {result['back_translation'][:50]}..." if len(result['back_translation']) > 50 else f"   Back:        {result['back_translation']}")
            lines.append("")
        
        return '\n'.join(lines)


# Load backward translators
print("Loading backward translators...")
translator_de_en = pipeline("translation", model="Helsinki-NLP/opus-mt-de-en")
translator_es_en = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")
translator_it_en = pipeline("translation", model="Helsinki-NLP/opus-mt-it-en")

# Create comparer
comparer = TranslationQualityComparer()
comparer.add_language_pair("French", translator_en_fr, translator_fr_en)
comparer.add_language_pair("German", translator_en_de, translator_de_en)
comparer.add_language_pair("Spanish", translator_en_es, translator_es_en)
comparer.add_language_pair("Italian", translator_en_it, translator_it_en)

print("Comparer ready!")

In [ ]:
# Test the comparer
test_texts = [
    "The new software update includes many security improvements.",
    "Break a leg at your presentation tomorrow!",
    "Our quarterly earnings exceeded analyst expectations.",
]

print("Translation Quality Comparison Across Languages:")
print("="*70)

for text in test_texts:
    comparison = comparer.compare_translations(text)
    print("\n" + comparer.format_comparison(comparison))

---

# Part 5: Mini-Project

## Project: Multilingual Content Adapter

**Scenario**: You're building a tool for a marketing team that needs to adapt promotional content for different markets. The tool should translate content, perform quality checks, and flag potential issues.

**Your goal**: Build a `MultilingualContentAdapter` class that:
1. Translates marketing copy to multiple languages
2. Performs back-translation quality checks
3. Flags potential localization issues
4. Provides confidence ratings for each translation

In [ ]:
# MINI-PROJECT: Multilingual Content Adapter
# ==========================================

class MultilingualContentAdapter:
    """
    Adapts marketing content for multiple language markets with quality checks.
    """
    
    # Common marketing terms that may need special attention
    MARKETING_TERMS = [
        "free", "sale", "discount", "limited time", "exclusive",
        "best", "#1", "guarantee", "special offer", "bonus"
    ]
    
    # Idioms that may not translate well
    COMMON_IDIOMS = [
        "game changer", "cutting edge", "best in class", "next level",
        "state of the art", "one stop shop", "bang for your buck",
        "raise the bar", "move the needle", "low hanging fruit"
    ]
    
    def __init__(self):
        self.markets = {}
    
    def add_market(self, market_name, language_code, forward_translator, backward_translator):
        """
        Add a target market with its translation pipelines.
        """
        self.markets[market_name] = {
            'language_code': language_code,
            'forward': forward_translator,
            'backward': backward_translator,
        }
    
    def detect_potential_issues(self, text):
        """
        Detect potential localization issues in the source text.
        """
        issues = []
        text_lower = text.lower()
        
        # Check for idioms
        for idiom in self.COMMON_IDIOMS:
            if idiom in text_lower:
                issues.append(f"Contains idiom '{idiom}' - may not translate directly")
        
        # Check for marketing superlatives
        superlatives = ["best", "#1", "number one", "leading", "top"]
        for term in superlatives:
            if term in text_lower:
                issues.append(f"Contains superlative '{term}' - legal restrictions may apply in some markets")
        
        # Check for currency symbols
        if '$' in text or '€' in text or '£' in text:
            issues.append("Contains currency symbol - prices may need market-specific adjustment")
        
        return issues
    
    def calculate_quality_score(self, original, back_translation):
        """
        Calculate a quality score based on back-translation.
        """
        orig_words = set(original.lower().replace('.', '').replace(',', '').replace('!', '').split())
        back_words = set(back_translation.lower().replace('.', '').replace(',', '').replace('!', '').split())
        
        intersection = len(orig_words & back_words)
        union = len(orig_words | back_words)
        
        return intersection / union if union > 0 else 0
    
    def adapt_content(self, content, content_type="general"):
        """
        Adapt content for all configured markets.
        
        Args:
            content: Original content (English)
            content_type: Type of content (general, headline, body, cta)
            
        Returns:
            dict with adaptations for each market
        """
        # Detect pre-translation issues
        pre_issues = self.detect_potential_issues(content)
        
        adaptations = []
        
        for market_name, market_config in self.markets.items():
            # Translate
            translation = market_config['forward'](content)[0]['translation_text']
            
            # Back-translate for quality check
            back_translation = market_config['backward'](translation)[0]['translation_text']
            
            # Calculate quality score
            quality_score = self.calculate_quality_score(content, back_translation)
            
            # Determine confidence level
            if quality_score >= 0.7:
                confidence = "High"
            elif quality_score >= 0.5:
                confidence = "Medium"
            else:
                confidence = "Low - Review Recommended"
            
            # Check for translation-specific issues
            translation_issues = []
            if quality_score < 0.5:
                translation_issues.append("Significant meaning change detected in back-translation")
            
            adaptations.append({
                'market': market_name,
                'language_code': market_config['language_code'],
                'translation': translation,
                'back_translation': back_translation,
                'quality_score': quality_score,
                'confidence': confidence,
                'issues': translation_issues,
            })
        
        # Sort by quality score
        adaptations.sort(key=lambda x: x['quality_score'], reverse=True)
        
        return {
            'original': content,
            'content_type': content_type,
            'pre_translation_issues': pre_issues,
            'adaptations': adaptations,
        }
    
    def format_report(self, result):
        """
        Format adaptation results as a readable report.
        """
        lines = []
        lines.append("╔" + "═"*68 + "╗")
        lines.append("║" + " MULTILINGUAL CONTENT ADAPTATION REPORT ".center(68) + "║")
        lines.append("╚" + "═"*68 + "╝")
        lines.append("")
        lines.append(f"Original ({result['content_type']}):")
        lines.append(f"  \"{result['original']}\"")
        lines.append("")
        
        if result['pre_translation_issues']:
            lines.append("⚠️  Pre-Translation Warnings:")
            for issue in result['pre_translation_issues']:
                lines.append(f"    • {issue}")
            lines.append("")
        
        lines.append("─" * 70)
        lines.append("MARKET ADAPTATIONS:")
        lines.append("─" * 70)
        
        for adaptation in result['adaptations']:
            confidence_icon = "✓" if adaptation['confidence'] == "High" else "◐" if adaptation['confidence'] == "Medium" else "⚠"
            
            lines.append(f"\n{confidence_icon} {adaptation['market']} ({adaptation['language_code']})")
            lines.append(f"  Quality: {adaptation['quality_score']:.0%} | Confidence: {adaptation['confidence']}")
            lines.append(f"  Translation: {adaptation['translation']}")
            lines.append(f"  Back-check:  {adaptation['back_translation']}")
            
            if adaptation['issues']:
                for issue in adaptation['issues']:
                    lines.append(f"  ⚠️ {issue}")
        
        return '\n'.join(lines)


# Create adapter
adapter = MultilingualContentAdapter()

# Add markets
adapter.add_market("France", "fr", translator_en_fr, translator_fr_en)
adapter.add_market("Germany", "de", translator_en_de, translator_de_en)
adapter.add_market("Spain", "es", translator_en_es, translator_es_en)
adapter.add_market("Italy", "it", translator_en_it, translator_it_en)

print("Multilingual Content Adapter initialized!")
print(f"Markets configured: {list(adapter.markets.keys())}")

In [ ]:
# Test with marketing content
marketing_content = [
    {
        'text': "Discover our game-changing productivity app - now 50% off!",
        'type': "headline"
    },
    {
        'text': "Join millions of satisfied customers who have transformed their workflow.",
        'type': "body"
    },
    {
        'text': "Start your free trial today - no credit card required!",
        'type': "cta"
    },
]

print("\n" + "="*70)
print(" MARKETING CONTENT LOCALIZATION ")
print("="*70)

for content in marketing_content:
    result = adapter.adapt_content(content['text'], content['type'])
    print("\n" + adapter.format_report(result))
    print("\n")

In [ ]:
# Test with content containing potential issues
tricky_content = "Our #1 rated cutting-edge solution is the best bang for your buck at just $9.99!"

result = adapter.adapt_content(tricky_content, "promotional")
print(adapter.format_report(result))

In [ ]:
# Try your own marketing content
# Uncomment and modify:

# my_content = "Your marketing text here"
# result = adapter.adapt_content(my_content, "general")
# print(adapter.format_report(result))

### Extension Ideas

If you want to extend this project further:

1. **Glossary support**: Add a glossary of terms that should be translated consistently
2. **Character limits**: Add support for social media character limits per market
3. **Tone adjustment**: Detect and report on tone differences across languages
4. **Export functionality**: Export translations to CSV or JSON for localization workflows
5. **Historical tracking**: Store previous translations for consistency across versions

---

# Part 6: Wrap-Up

## Key Takeaways

1. **Translation uses encoder-decoder architecture** - Same as summarization, but converts between languages instead of compressing

2. **Language pairs matter** - Each direction (en→fr vs fr→en) typically needs a separate model, or use multilingual models

3. **Back-translation** is a useful quality check - Translate back to source to detect meaning loss

4. **Idioms and cultural content** are challenging - Literal translations often miss the meaning

5. **BLEU scores** measure translation quality by comparing to human references

6. **Domain matters** - General models may struggle with technical, legal, or specialized content

## Common Mistakes to Avoid

| Mistake | Why It's a Problem |
|---------|-------------------|
| Assuming translations are perfect | Machine translation still makes errors |
| Ignoring cultural context | Direct translation may be inappropriate |
| Not quality-checking idioms | Figurative language often translates literally |
| Using wrong language pair direction | en→fr model won't work for fr→en |
| Skipping back-translation checks | Meaning loss may go undetected |

## What's Next?

In **Notebook 8: Text Similarity with Embeddings**, you'll learn:
- How models represent text as numerical vectors
- Using embeddings to find similar content
- Cosine similarity and different pooling strategies

Embeddings build on translation concepts by showing how models represent meaning in vector space!

---

## Solutions

### Check Your Understanding (Quiz Answers)

1. **C) Encoder-decoder (like T5, BART)** - Translation needs to encode source meaning and decode into target language
2. **B) Capture the meaning of source text** - The encoder creates a meaning representation that the decoder uses
3. **B) A source and target language combination** - Like en-fr (English to French)
4. **C) Both A and B** - Back-translation is used for quality checking AND as a data augmentation technique for training

### Exercise 2: Key Insights

In [ ]:
# Key insights from translation quality analysis:

# 1. Back-translation reveals meaning loss:
#    - High similarity (>70%) = Good translation
#    - Medium similarity (50-70%) = Acceptable but review
#    - Low similarity (<50%) = Likely issues

# 2. Idioms are red flags:
#    - "Break a leg" → "Casse-toi une jambe" (literal, wrong!)
#    - Should be: "Bonne chance" or "Merde" (French equivalent)

# 3. Context matters:
#    - "Bank" could be financial or riverside
#    - Longer context helps disambiguate

# 4. Domain-specific content needs care:
#    - Technical terms may not translate
#    - Legal/medical content should be professionally reviewed

quality_guide = {
    'High (70%+)': 'Ready for use with light review',
    'Medium (50-70%)': 'Needs human review',
    'Low (<50%)': 'Significant editing needed',
}

print("Translation Quality Guide:")
print("="*50)
for level, action in quality_guide.items():
    print(f"  {level:20s} → {action}")

---

## Additional Resources

- [Hugging Face Translation Task](https://huggingface.co/tasks/translation)
- [Helsinki-NLP Models](https://huggingface.co/Helsinki-NLP) - Hundreds of language pairs
- [BLEU Score Explained](https://en.wikipedia.org/wiki/BLEU) - Standard translation metric
- [Google's Neural Machine Translation](https://arxiv.org/abs/1609.08144) - Foundational paper
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762) - Transformer architecture paper
- [No Language Left Behind](https://ai.meta.com/research/no-language-left-behind/) - Meta's multilingual model